# LSTM
**Author:** [Divyanshu Raghuwanshi](https://www.linkedin.com/in/divyanshu-raghuwanshi-85037b160/)<br>
**Date created:** 2020/17/05<br>

In [1]:
# lstm model
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM
from keras.utils import to_categorical
from matplotlib import pyplot
import numpy as np
import pandas as pd

Using TensorFlow backend.


In [20]:
homepath = 'D:/kit.me/LSTM Codes and Data/Summary_Data'

In [21]:
def load_dataset(homepath):
    # load all train
    trainX = np.load(homepath+'/Train_data.npy')
    trainy = np.load(homepath+'/Train_labels.npy')
    testX = np.load(homepath+'/Test_data.npy')
    testy = np.load(homepath+'/Test_labels.npy')

    trainy = trainy - 1
    testy = testy - 1
    # one hot encode y
    trainy = to_categorical(trainy)
    testy = to_categorical(testy)
    print(trainX.shape, trainy.shape, testX.shape, testy.shape)
    return trainX, trainy, testX, testy

In [22]:
trainX, trainy, testX, testy = load_dataset(homepath)

(865, 300, 16) (865, 5) (217, 300, 16) (217, 5)


In [23]:
def evaluate_model(trainX, trainy, testX, testy):
	verbose, epochs, batch_size = 0, 15, 64
	n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainy.shape[1]
	model = Sequential()
	model.add(LSTM(100, input_shape=(n_timesteps,n_features)))
	model.add(Dropout(0.5))
	model.add(Dense(100, activation='relu'))
	model.add(Dense(n_outputs, activation='softmax'))
	model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
	# fit network
	model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
	# evaluate model
	_, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
	return accuracy

In [34]:
def summarize_results(scores):
    print(scores)
    m, s = mean(scores), std(scores)
    print('Accuracy: %.3f%% (+/-%.3f)' % (m, s))
    
    fullCatWriter = pd.ExcelWriter(homepath + '/Result.xlsx', engine='xlsxwriter')     
    dataframe1 = pd.DataFrame(scores)            
    dataframe1.to_excel(fullCatWriter, sheet_name='Full Raw', startrow=0 , startcol=0)            
    fullCatWriter.save()      
    fullCatWriter.close() 


In [35]:
def run_experiment(repeats=10):
	# load data
	trainX, trainy, testX, testy = load_dataset(homepath)
	# repeat experiment
	scores = list()
	for r in range(repeats):
		score = evaluate_model(trainX, trainy, testX, testy)
		score = score * 100.0
		print('>#%d: %.3f' % (r+1, score))
		scores.append(score)
	# summarize results
	summarize_results(scores)


In [36]:
run_experiment()

(4338, 120, 16) (4338, 5) (1085, 120, 16) (1085, 5)
>#1: 99.908
>#2: 99.908
>#3: 99.816
>#4: 99.539
>#5: 99.816
>#6: 99.171
>#7: 99.724
>#8: 99.631
>#9: 99.816
>#10: 99.908
[99.90783410138249, 99.90783410138249, 99.81566820276497, 99.53917050691244, 99.81566820276497, 99.17050691244239, 99.72350230414746, 99.63133640552996, 99.81566820276497, 99.90783410138249]
Accuracy: 99.724% (+/-0.218)
